# Analítica Avanzada de Datos.
---

 FECHA DE ENTREGA: **Miercoles 12 de Abril**

## Práctica 2. Regresión Lineal


 Utilizando el dataset *Air Quality Dataset* (AirQualityUCI.csv) y cuya descripción es la siguiente:

El dataset contiene **9357** instancias de respuestas promediadas cada hora de un conjunto de *5 sensores químicos* de óxido metálico integrados en un dispositivo multisensor químico de calidad del aire. El dispositivo estaba situado sobre el terreno en una zona significativamente contaminada, a nivel de carretera, dentro de una ciudad italiana. Los datos se registraron de marzo de 2004 a febrero de 2005 (un año), lo que representa el periodo más largo de registros disponibles de respuestas de dispositivos de sensores químicos de calidad del aire desplegados sobre el terreno. Las concentraciones medias horarias reales de CO, hidrocarburos no metánicos, benceno, óxidos de nitrógeno (NOx) y dióxido de nitrógeno (NO2) fueron proporcionadas por un analizador certificado de referencia situado en el mismo lugar. Existen evidencias de sensibilidades cruzadas, así como de desviaciones del concepto y del sensor, tal como se describe en De Vito et al., Sens. And Act. B, Vol. 129,2,2008 (cita requerida) afectando eventualmente a las capacidades de estimación de concentración de los sensores. Los **valores perdidos** se marcan con el valor **-200**.

https://archive.ics.uci.edu/ml/datasets/Air+quality

Sus atributos son:

0. Fecha (DD/MM/AAAA)
1. Hora (HH.MM.SS)
2. Concentración media horaria real de CO en mg/m^3 (analizador de referencia)
3. Respuesta media horaria del sensor PT08.S1 (óxido de estaño) (nominalmente dirigido a CO)
4. Promedio horario real de la concentración total de hidrocarburos no metánicos en microg/m^3 (analizador de referencia)
5. Concentración media horaria real de benceno en microg/m^3 (analizador de referencia)
6. Respuesta media horaria del sensor PT08.S2 (titania) (objetivo nominal NMHC)
7. Concentración de NOx media horaria real en ppb (analizador de referencia)
8. PT08.S3 (óxido de tungsteno) respuesta media horaria del sensor (objetivo nominal NOx)
9. Concentración de NO2 real media horaria en microg/m^3 (analizador de referencia)
10. PT08.S4 (óxido de tungsteno) respuesta media horaria del sensor (objetivo nominal NO2)
11. PT08.S5 (óxido de indio) respuesta media horaria del sensor (objetivo nominal O3)
12. Temperatura en Â°C
13. Humedad relativa (%)
14. AH Humedad absoluta

Utilizando este Dataset y los modelos de regresión vistos en la unidad, contesta la siguiente pregunta:
- **¿En que horarios se tiene la mayor contaminación del aire?**

1. Debes analizar:
    - Si existen valores perdidos
    - Elegir las características más apropiadas
    - Calcular la correlación estadística

2. Con base en tu análisis previo, entrena el modelo de regresión (hold out 70% - 30%)
3. Evalua tu modelo entrenado calculando:
    - MSE
    - RMSE
    - R-squared
4. Escribe una breve conclusión de los resultados de tu análisis completo

In [2]:
# Librerias a ocupar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

### Preprosesamiento

In [17]:
# leemos dataset
df = pd.read_csv('AirQualityUCI.csv', sep=',')

# Visualizamos el dataset
df.head(5)

# Comprobamos el total de valores perdidos
# df.isnull().sum()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [19]:
# Comprobamos la cantidad de nulos
df.isnull().sum()

Date              114
Time              114
CO(GT)            114
PT08.S1(CO)       114
NMHC(GT)          114
C6H6(GT)          114
PT08.S2(NMHC)     114
NOx(GT)           114
PT08.S3(NOx)      114
NO2(GT)           114
PT08.S4(NO2)      114
PT08.S5(O3)       114
T                 114
RH                114
AH                114
Unnamed: 15      9471
Unnamed: 16      9471
dtype: int64

In [21]:
df.dtypes

Date              object
Time              object
CO(GT)           float64
PT08.S1(CO)      float64
NMHC(GT)         float64
C6H6(GT)         float64
PT08.S2(NMHC)    float64
NOx(GT)          float64
PT08.S3(NOx)     float64
NO2(GT)          float64
PT08.S4(NO2)     float64
PT08.S5(O3)      float64
T                float64
RH               float64
AH               float64
Unnamed: 15      float64
Unnamed: 16      float64
dtype: object

In [24]:
def simulate_moving_averages_process(
    parameters: List[float]=[0.9], 
    timesteps: int=100, 
    mean: float=0.,
    std: float=1.a
    ):
    
    '''
    This function simulates an MA(q) process with mean 0.
    
    Inputs:
        parameters: A list of the parameters theta used to compute the moving averages simulation
        timesteps: The number of timesteps to simulate.
        mean: The mean used to sample the white noise
        std: The standard deviation used to sample the white noise
    Output:
        The time series simulation
    '''
    # Genera una serie de ruido blanco
    white_noise = np.random.normal(loc=mean, scale=std, size=timesteps)

    # Inicializa la serie de tiempo simulada con ceros
    ma_process = np.zeros(timesteps)

    # Calcula los valores de la serie de tiempo simulando la media movil
    for i in range(len(ma_process)):
        for j in range(len(parameters)):
            if i - j >= 0:
                ma_process[i] += parameters[j] * white_noise[i - j]
    
    return ma_process
    
    return np.zeros(timesteps)

-200.0    1639
 89.0       41
 65.0       37
 93.0       36
 122.0      36
          ... 
 791.0       1
 674.0       1
 844.0       1
 809.0       1
 594.0       1
Name: NOx(GT), Length: 926, dtype: int64